In [ ]:
import pandas as pd

In [ ]:
# reading the data from the csv file
data = pd.read_csv('emotions.csv')
X = data.drop("label", axis=1)

In [ ]:
# shape of data
data.shape

In [ ]:
# data example
data.head()

In [ ]:
# encoding the class from labels to numbers (0, 1, 2)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['label']=le.fit_transform(data['label'])

In [ ]:
# array only for the class
y = data['label']

In [ ]:
from matplotlib import pyplot

In [ ]:
# create a subplot for each time series
values = data.loc[:,'fft_0_b':'fft_9_b'].values
pyplot.figure()
for i in range(values.shape[1]):
    pyplot.subplot(values.shape[1], 1, i+1)
    pyplot.plot(values[:, i])
pyplot.show()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# calling the train function and spliting the data to train and test sets randomly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1/3, random_state=40)

In [ ]:
# transforming the class labels from numbers to binary
# example:                     0 1 2
# 1009    2             1009   0 0 1
# 1150    1      ->     1150   0 1 0
# 1460    0             1460   1 0 0
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
# categories distribution
y_test.value_counts()

------------------------------------- MultiLayer Perceptron ------------------------------------------

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

test_k_values = range(1, 261)
loss = []
test_k_values_accuracy = []
train_k_values_accuracy = []

# calling the classifier
test_mlp = MLPClassifier(hidden_layer_sizes=(7000,100), activation='logistic', solver='sgd',
            learning_rate_init=0.01, learning_rate='adaptive', max_iter=260, verbose=False,
            random_state=1, alpha=0.01, n_iter_no_change=10, tol=1e-4, batch_size=30)

# evaluating accuracy and loss
for k in test_k_values:
    loss.append(test_mlp.partial_fit(X_train, y_train, np.unique(y)).loss_)
    test_k_values_accuracy.append(test_mlp.score(X_test, y_test))
    train_k_values_accuracy.append(test_mlp.score(X_train, y_train))
    k=k+1

# summarize history for accuracy
plt.plot(test_k_values, train_k_values_accuracy, '-', label="Training Accuracy")
plt.plot(test_k_values, test_k_values_accuracy, ':', label="Validation Accuracy")
plt.title('MLP - Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()

# summarize history for loss
plt.plot(test_k_values, loss, '-', label="Loss")
plt.title('MLP - Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# prediction of the algorithm form the training set
y_pred = test_mlp.predict(X_train)
model_acc_train = accuracy_score(y_train, y_pred)

In [ ]:
# prediction of the algorithm form the testing set
y_pred = test_mlp.predict(X_test)
model_acc_test = accuracy_score(y_test, y_pred)

In [ ]:
print("MLP - Training Accuracy: %.1f" %  (model_acc_train*100))
print("MLP - Testing Accuracy: %.1f" % (model_acc_test*100))
print("MLP - Loss: %.2f" % loss[-1])

------------------------------------------- Keras Model -----------------------------------------------

In [ ]:
# determine the number of input features
Keras_n_features = X_train.shape

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Flatten

# define Keras model
Keras_model = Sequential()
# adding dense layers
Keras_model.add(Dense(512, input_shape=Keras_n_features[1:]))
Keras_model.add(Dense(512, activation='sigmoid'))
Keras_model.add(Dense(512, activation='sigmoid'))
# adding output layer
Keras_model.add(Dense(3, activation='softmax'))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# calculating early stopping and learning rate scheduler
Keras_es = EarlyStopping(monitor='val_loss', mode='min', verbose=10, patience=10)
Keras_lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch:0.01 * np.exp(-epoch / 10.))

In [ ]:
from tensorflow.keras import optimizers

# compiling the model
Keras_opt_adam = optimizers.Adam(learning_rate=0.01)
Keras_model.compile(optimizer=Keras_opt_adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# model summary
Keras_model.summary()

In [ ]:
# model image
tf.keras.utils.plot_model(Keras_model)

In [ ]:
# training the model
Keras_hist = Keras_model.fit(X_train, y_train, epochs=30, batch_size=50, verbose=0, 
                             validation_data=(X_test, y_test), callbacks=[Keras_es, Keras_lr_schedule])

In [ ]:
import seaborn as sns
sns.set()
 
Keras_acc = Keras_hist.history['accuracy']
Keras_val = Keras_hist.history['val_accuracy']
Keras_epochs = range(1, len(Keras_acc) + 1)

# summarize history for accuracy
plt.plot(Keras_epochs, Keras_acc, '-', label='Training accuracy')
plt.plot(Keras_epochs, Keras_val, ':', label='Validation accuracy')
plt.title('Keras - Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()

# summarize history for loss
Keras_loss = Keras_hist.history['loss']
Keras_val_loss = Keras_hist.history['val_loss']

plt.plot(Keras_epochs, Keras_loss, '-', label='Loss')
plt.plot(Keras_epochs, Keras_val_loss, ':', label='Validation loss')
plt.title('Keras - Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

In [ ]:
# evaluating the model
Keras_model_acc_train = Keras_model.evaluate(X_train, y_train,verbose=0)
Keras_model_acc_test = Keras_model.evaluate(X_test, y_test,verbose=0)

In [ ]:
print("Keras - Training Accuracy: %.1f" % (Keras_model_acc_train[1]*100))
print("Keras - Testing Accuracy: %.1f" % (Keras_model_acc_test[1]*100))
print("Keras - Loss: %.2f" % Keras_loss[-1])

------------------------------------- Long Short-Term Memory -----------------------------------------

In [ ]:
# Expanding the dimensions from 2D to 3D as needed for the LSTM, GRU amd CNN models
X_train = np.array(X_train).reshape((X_train.shape[0],X_train.shape[1],1))
X_test = np.array(X_test).reshape((X_test.shape[0],X_test.shape[1],1))

In [ ]:
# Determine the number of input features
n_features = X_train.shape

In [ ]:
from tensorflow.keras.layers import LSTM

# define LSTM model
LSTM_model = Sequential()
# adding LSTM layer
LSTM_model.add(LSTM(3, input_shape=n_features[1:], return_sequences=True))
# adding fully connected layer
LSTM_model.add(Flatten())
# adding output layer
LSTM_model.add(Dense(3, activation='softmax'))

In [ ]:
# calculating early stopping and learning rate scheduler
LSTM_es = EarlyStopping(monitor='val_loss', mode='min', verbose=10, patience=10)
LSTM_lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))

In [ ]:
# compiling the model
LSTM_opt_adam = optimizers.Adam(learning_rate=0.001)
LSTM_model.compile(optimizer=LSTM_opt_adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# model summary
LSTM_model.summary()

In [ ]:
# model image
tf.keras.utils.plot_model(LSTM_model)

In [ ]:
# training the model
LSTM_hist = LSTM_model.fit(X_train, y_train, epochs=5, batch_size=50, verbose=0, 
                 validation_data=(X_test, y_test), callbacks=[LSTM_es, LSTM_lr_schedule])

In [ ]:
LSTM_acc = LSTM_hist.history['accuracy']
LSTM_val = LSTM_hist.history['val_accuracy']
LSTM_epochs = range(1, len(LSTM_acc) + 1)

# summarize hisrory for accuracy
plt.plot(LSTM_epochs, LSTM_acc, '-', label='Training accuracy')
plt.plot(LSTM_epochs, LSTM_val, ':', label='Validation accuracy')
plt.title('LSTM - Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()

# summarize history for loss
LSTM_loss = LSTM_hist.history['loss']
LSTM_val_loss = LSTM_hist.history['val_loss']

plt.plot(LSTM_epochs, LSTM_loss, '-', label='Loss')
plt.plot(LSTM_epochs, LSTM_val_loss, ':', label='Validation loss')
plt.title('LSTM - Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

In [ ]:
# evaluating the model
LSTM_model_acc_train = LSTM_model.evaluate(X_train, y_train, verbose=0)
LSTM_model_acc_test = LSTM_model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print("LSTM - Training Accuracy: %.1f" %  (LSTM_model_acc_train[1]*100))
print("LSTM - Test Accuracy: %.1f" %  (LSTM_model_acc_test[1]*100))
print("LSTM - Loss: %.2f" % LSTM_loss[-1])

-------------------------------------- Gated Recurrent Unit ------------------------------------------

In [ ]:
from tensorflow.keras.layers import GRU

# define GRU model
GRU_model = Sequential()
# adding GRU layer
GRU_model.add(GRU(200, input_shape=n_features[1:]))
# adding fully connected layer
GRU_model.add(Flatten())
# adding output layer
GRU_model.add(Dense(3, activation="softmax"))

In [ ]:
# calculating early stopping and learning rate scheduler
GRU_es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
GRU_lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))

In [ ]:
# compiling the model
GRU_opt_adam = optimizers.Adam(learning_rate=0.001)
GRU_model.compile(optimizer=GRU_opt_adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# model summary
GRU_model.summary()

In [ ]:
# model image
tf.keras.utils.plot_model(GRU_model)

In [ ]:
# training the model
GRU_hist = GRU_model.fit(X_train, y_train, epochs=40, batch_size=20, verbose=0, 
                 validation_data=(X_test, y_test), callbacks=[GRU_es, GRU_lr_schedule])

In [ ]:
GRU_acc = GRU_hist.history['accuracy']
GRU_val = GRU_hist.history['val_accuracy']
GRU_epochs = range(1, len(GRU_acc) + 1)

# summarize history for accuracy
plt.plot(GRU_epochs, GRU_acc, '-', label='Training accuracy')
plt.plot(GRU_epochs, GRU_val, ':', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()

# summarize history for loss
GRU_loss = GRU_hist.history['loss']
GRU_val_loss = GRU_hist.history['val_loss']

plt.plot(GRU_epochs, GRU_loss, '-', label='Loss')
plt.plot(GRU_epochs, GRU_val_loss, ':', label='Validation loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

In [ ]:
# evaluating the model
GRU_model_acc_train = GRU_model.evaluate(X_train, y_train, verbose=0)
GRU_model_acc_test = GRU_model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print("GRU - Training Accuracy: %.1f" % (GRU_model_acc_train[1]*100))
print("GRU - Testing Accuracy: %.1f" % (GRU_model_acc_test[1]*100))
print("GRU - Loss: %.2f" % GRU_loss[-1])

---------------------------------- Convolutional Neural Network --------------------------------------

In [ ]:
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D

# define CNN model
CNN_model = Sequential()
# adding convolution layer
CNN_model.add(Conv1D(128, kernel_size=4, activation='sigmoid', input_shape=n_features[1:], padding='same'))
# adding pooling layer
CNN_model.add(MaxPool1D(pool_size=2, padding='same'))
# adding fully connected layer
CNN_model.add(Flatten())
# adding output layer
CNN_model.add(Dense(3, activation='softmax'))

In [ ]:
# calculating early stopping and learning rate scheduler
CNN_es = EarlyStopping(monitor='val_loss', mode='min', verbose=10, patience=10)
CNN_lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))

In [ ]:
# compiling the model
CNN_opt_adam = optimizers.Adam(learning_rate=0.001)
CNN_model.compile(optimizer=CNN_opt_adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# model summary
CNN_model.summary()

In [ ]:
# model image
tf.keras.utils.plot_model(CNN_model)

In [ ]:
# fit the model
CNN_hist = CNN_model.fit(X_train, y_train, epochs=8, batch_size=35, verbose=0, 
                 validation_data=(X_test, y_test), callbacks=[CNN_es, CNN_lr_schedule])

In [ ]:
CNN_acc = CNN_hist.history['accuracy']
CNN_val = CNN_hist.history['val_accuracy']
CNN_epochs = range(1, len(CNN_acc) + 1)

# summarize history for accuracy
plt.plot(CNN_epochs, CNN_acc, '-', label='Training accuracy')
plt.plot(CNN_epochs, CNN_val, ':', label='Validation accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='best')
plt.show()

# summarize history for loss
CNN_loss = CNN_hist.history['loss']
CNN_val_loss = CNN_hist.history['val_loss']

plt.plot(CNN_epochs, CNN_loss, '-', label='Loss')
plt.plot(CNN_epochs, CNN_val_loss, ':', label='Validation loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='best')
plt.show()

In [ ]:
# evaluating the model
CNN_model_acc_train = CNN_model.evaluate(X_train, y_train, verbose=0)
CNN_model_acc_test = CNN_model.evaluate(X_test, y_test, verbose=0)

In [ ]:
print("CNN - Training Accuracy: %.1f" % (CNN_model_acc_train[1]*100))
print("CNN - Testing Accuracy: %.1f" % (CNN_model_acc_test[1]*100))
print("CNN - Loss: %.2f" % CNN_loss[-1])